# Test lógico negación

In [62]:
import sys

sys.path.append("../src")

In [63]:
import nltk
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm
from pyprover import *

from nltk.sem.logic import LogicParser

lp = LogicParser()

In [64]:
from config.config import PATHS
from parser.parser_utils import Parse2pyprover

In [71]:
data_folder = PATHS['training_data_folder']
data_path = Path(data_folder, 'negacion_5_fol.csv')
data = pd.read_csv(data_path, names=['Sentence 1', 'Sentence 2', 'Relation'], sep=";", nrows=181)
print(f'Data shape: {data.shape}')
data.tail()

Data shape: (181, 3)


,Sentence 1,Sentence 2,Relation
176,(exists x.JINETE(x) & all x.(JINETE(x) -> (ACO...,exists x.(JINETE(x) & -ACORDAR(x) & -ENTRAR(x)),1
177,(exists x.JINETE(x) & all x.(JINETE(x) -> (ACO...,exists x.(JINETE(x) & -ACORDAR(x) & -DEAMBULAR...,1
178,(exists x.JINETE(x) & all x.(JINETE(x) -> (DEA...,exists x.(JINETE(x) & -DEAMBULAR(x) & -ENTRAR(x)),1
179,(exists x.JINETE(x) & all x.(JINETE(x) -> (DEA...,exists x.(JINETE(x) & -DEAMBULAR(x) & -ACORDAR...,1
180,-(exists x.ABUELO(x) & all x.(ABUELO(x) -> ENT...,(exists x.ABUELO(x) & all x.(ABUELO(x) -> ENTR...,1


In [72]:
sentences1 = data['Sentence 1'].tolist()
sentences2 = data['Sentence 2'].tolist()

In [73]:
def test_negacion(sentence1:str, sentence2:str) -> bool:
    '''
    Test negation between two sentences.
    '''
    #Test sentence1 implies -sentence2
    sentence1_nltk = lp.parse(f'{sentence1}')
    # print(f'Sentence 1 NLTK: {sentence1_nltk}')
    sentence1_pyprover = Parse2pyprover.parse(sentence1_nltk)

    negated_sentence2_nltk = lp.parse(f'-{sentence2}')
    # print(f'Negación Sentence 2 NLTK: {negated_sentence2_nltk}')
    negated_sentence2_pyprover = Parse2pyprover.parse(negated_sentence2_nltk)

    premisas = [sentence1_pyprover]
    conclusion = negated_sentence2_pyprover
    resultado1 = proves(premisas, conclusion)
    # print('Resultado:', resultado1)

    # Test -sentence1 implies sentence2
    sentence2_nltk = lp.parse(f'{sentence2}')
    # print(f'Sentence 2 NLTK: {sentence2_nltk}')
    sentence2_pyprover = Parse2pyprover.parse(sentence2_nltk)

    negated_sentence1_nltk = lp.parse(f'-{sentence1}')
    # print(f'Negación Sentence 1 NLTK: {negated_sentence1_nltk}')
    negated_sentence1_pyprover = Parse2pyprover.parse(negated_sentence1_nltk)


    premisas = [sentence2_pyprover]
    conclusion = negated_sentence1_pyprover
    resultado2 = proves(premisas, conclusion)
    # print('Resultado:', resultado2)

    return resultado1 and resultado2

In [74]:
tested = list()
for index, row in data.iterrows():
    sentences1 = row['Sentence 1']
    sentences2 = row['Sentence 2']
    print('Sentences:', sentences1, sentences2)
    relation = row['Relation']
    result = test_negacion(sentences1, sentences2)
    tested.append(int(result))

data['Tested'] = tested
data.head()

Sentences: (exists x.ABUELO(x) & all x.(ABUELO(x) -> ENTRAR(x))) exists x.(ABUELO(x) & -ENTRAR(x))
Sentences: (exists x.ABUELO(x) & all x.(ABUELO(x) -> ACORDAR(x))) exists x.(ABUELO(x) & -ACORDAR(x))
Sentences: (exists x.ABUELO(x) & all x.(ABUELO(x) -> DEAMBULAR(x))) exists x.(ABUELO(x) & -DEAMBULAR(x))
Sentences: (exists x.ZORRO(x) & all x.(ZORRO(x) -> ENTRAR(x))) exists x.(ZORRO(x) & -ENTRAR(x))
Sentences: (exists x.ZORRO(x) & all x.(ZORRO(x) -> ACORDAR(x))) exists x.(ZORRO(x) & -ACORDAR(x))
Sentences: (exists x.ZORRO(x) & all x.(ZORRO(x) -> DEAMBULAR(x))) exists x.(ZORRO(x) & -DEAMBULAR(x))
Sentences: (exists x.JINETE(x) & all x.(JINETE(x) -> ENTRAR(x))) exists x.(JINETE(x) & -ENTRAR(x))
Sentences: (exists x.JINETE(x) & all x.(JINETE(x) -> ACORDAR(x))) exists x.(JINETE(x) & -ACORDAR(x))
Sentences: (exists x.JINETE(x) & all x.(JINETE(x) -> DEAMBULAR(x))) exists x.(JINETE(x) & -DEAMBULAR(x))
Sentences: (exists x.(ABUELO(x) & RÁPIDO(x)) & all x.((ABUELO(x) & RÁPIDO(x)) -> ENTRAR(x))) e

ParseException: Expected {top | ⊤ | ┬ | true | bot | ⊥ | ┴ | _\|_ | false | {Suppress:(EX | ∃ | exists | ex | Ex | E | TE | te | Te) Re:("[a-z0-9_]\w*'*") [colon prop] {Suppress:('.') | Suppress:(',')} Forward: expr} | {Suppress:(FA | ∀ | fa | forall | A | Fa) Re:("[a-z0-9_]\w*'*") [colon prop] {Suppress:('.') | Suppress:(',')} Forward: expr} | {Re:("[A-Z]\w*'*") Suppress:('(') - {Forward: {Re:("[a-z0-9_]\w*'*") Suppress:('(') - {Forward: func | Re:("[a-z0-9_]\w*'*") | {Suppress:('?') Re:("[a-z0-9_]\w*'*")}} [{comma term}]... [comma] - Suppress:(')')} | Re:("[a-z0-9_]\w*'*") | {Suppress:('?') Re:("[a-z0-9_]\w*'*")}} [{comma term}]... [comma] - Suppress:(')')} | Re:("[A-Z]\w*'*") | {Forward: {Re:("[a-z0-9_]\w*'*") Suppress:('(') - {Forward: func | Re:("[a-z0-9_]\w*'*") | {Suppress:('?') Re:("[a-z0-9_]\w*'*")}} [{comma term}]... [comma] - Suppress:(')')} | Re:("[a-z0-9_]\w*'*") | {Suppress:('?') Re:("[a-z0-9_]\w*'*")}} Suppress:('=') {Forward: {Re:("[a-z0-9_]\w*'*") Suppress:('(') - {For

In [69]:
data[data['Tested'] != data['Relation']]

,Sentence 1,Sentence 2,Relation,Tested


In [70]:
data.tail()

,Sentence 1,Sentence 2,Relation,Tested
175,(exists x.JINETE(x) & all x.(JINETE(x) -> (ENT...,exists x.(JINETE(x) & -ENTRAR(x) & -DEAMBULAR(x)),1,1
176,(exists x.JINETE(x) & all x.(JINETE(x) -> (ACO...,exists x.(JINETE(x) & -ACORDAR(x) & -ENTRAR(x)),1,1
177,(exists x.JINETE(x) & all x.(JINETE(x) -> (ACO...,exists x.(JINETE(x) & -ACORDAR(x) & -DEAMBULAR...,1,1
178,(exists x.JINETE(x) & all x.(JINETE(x) -> (DEA...,exists x.(JINETE(x) & -DEAMBULAR(x) & -ENTRAR(x)),1,1
179,(exists x.JINETE(x) & all x.(JINETE(x) -> (DEA...,exists x.(JINETE(x) & -DEAMBULAR(x) & -ACORDAR...,1,1
